In [1]:
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import numpy as np

# Load Pune boundary
pune_boundary = gpd.read_file(r"C:\Users\admin\Downloads\pune-admin-wards.geojson")

In [2]:
# Define grid spacing and create a grid of points within the boundary
grid_spacing = 0.01  # Adjust this for the density of points
minx, miny, maxx, maxy = pune_boundary.total_bounds
x_coords = np.arange(minx, maxx, grid_spacing)
y_coords = np.arange(miny, maxy, grid_spacing)
grid_points = [Point(x, y) for x in x_coords for y in y_coords]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs=pune_boundary.crs)
grid_gdf = gpd.clip(grid_gdf, pune_boundary)

# Define the OSM tags to extract relevant data
tags = {
    "amenity": ["hospital", "school", "police", "cinema", "fuel", "parking"],
    "leisure": ["park"],
    "building": ["commercial", "residential"],
    "shop": True,
    "landuse": ["residential", "industrial", "commercial"],
    "power": "generator",
    "public_transport": "station",
    "man_made": "wastewater_plant"
}

In [3]:
# Extract features from OSM within Pune boundary
all_features = ox.geometries_from_polygon(pune_boundary.geometry[0], tags=tags)

# Separate each category of interest for individual distance calculations
hospitals = all_features[all_features["amenity"] == "hospital"]
schools = all_features[all_features["amenity"] == "school"]
police_stations = all_features[all_features["amenity"] == "police"]
cinemas = all_features[all_features["amenity"] == "cinema"]
fuel_stations = all_features[all_features["amenity"] == "fuel"]
parking_areas = all_features[all_features["amenity"] == "parking"]
parks = all_features[all_features["leisure"] == "park"]
commercial_buildings = all_features[all_features["building"] == "commercial"]
residential_buildings = all_features[all_features["building"] == "residential"]
retail_outlets = all_features[all_features["shop"].notnull()]
residential_landuse = all_features[all_features["landuse"] == "residential"]
industrial_landuse = all_features[all_features["landuse"] == "industrial"]
commercial_landuse = all_features[all_features["landuse"] == "commercial"]
power_generators = all_features[all_features["power"] == "generator"]
transit_stations = all_features[all_features["public_transport"] == "station"]
wastewater_plants = all_features[all_features["man_made"] == "wastewater_plant"]

C:\Users\admin\AppData\Local\Temp\ipykernel_15380\1705666532.py:2: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  all_features = ox.geometries_from_polygon(pune_boundary.geometry[0], tags=tags)


In [4]:
# Helper function to calculate minimum distance from each grid point to a feature
def calculate_min_distance(grid, features):
    if features.empty:
        return np.nan
    return grid.geometry.apply(lambda point: features.distance(point).min())

# Calculate distances for each feature category
grid_gdf["distance_to_hospital"] = calculate_min_distance(grid_gdf, hospitals)
grid_gdf["distance_to_school"] = calculate_min_distance(grid_gdf, schools)
grid_gdf["distance_to_police_station"] = calculate_min_distance(grid_gdf, police_stations)
grid_gdf["distance_to_cinema"] = calculate_min_distance(grid_gdf, cinemas)
grid_gdf["distance_to_fuel_station"] = calculate_min_distance(grid_gdf, fuel_stations)
grid_gdf["distance_to_parking"] = calculate_min_distance(grid_gdf, parking_areas)
grid_gdf["distance_to_park"] = calculate_min_distance(grid_gdf, parks)
grid_gdf["distance_to_commercial_building"] = calculate_min_distance(grid_gdf, commercial_buildings)
grid_gdf["distance_to_residential_building"] = calculate_min_distance(grid_gdf, residential_buildings)
grid_gdf["distance_to_retail"] = calculate_min_distance(grid_gdf, retail_outlets)
grid_gdf["distance_to_residential_landuse"] = calculate_min_distance(grid_gdf, residential_landuse)
grid_gdf["distance_to_industrial_landuse"] = calculate_min_distance(grid_gdf, industrial_landuse)
grid_gdf["distance_to_commercial_landuse"] = calculate_min_distance(grid_gdf, commercial_landuse)
grid_gdf["distance_to_power_generator"] = calculate_min_distance(grid_gdf, power_generators)
grid_gdf["distance_to_transit_station"] = calculate_min_distance(grid_gdf, transit_stations)
grid_gdf["distance_to_wastewater_plant"] = calculate_min_distance(grid_gdf, wastewater_plants)

# View the first few rows of grid_gdf to confirm distances
grid_gdf.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_15380\2790424431.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return grid.geometry.apply(lambda point: features.distance(point).min())
C:\Users\admin\AppData\Local\Temp\ipykernel_15380\2790424431.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return grid.geometry.apply(lambda point: features.distance(point).min())
C:\Users\admin\AppData\Local\Temp\ipykernel_15380\2790424431.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return grid.geometry.apply(lambda point: features.distance(point).min())
C:\Users\admin\AppD

,geometry,distance_to_hospital,distance_to_school,distance_to_police_station,distance_to_cinema,distance_to_fuel_station,distance_to_parking,distance_to_park,distance_to_commercial_building,distance_to_residential_building,distance_to_retail,distance_to_residential_landuse,distance_to_industrial_landuse,distance_to_commercial_landuse,distance_to_power_generator,distance_to_transit_station,distance_to_wastewater_plant
182,POINT (73.84156 18.44883),0.109201,0.089969,0.093147,0.118434,0.094687,0.091997,0.085115,0.106410,0.098904,0.094439,0.085823,0.097652,0.096227,0.108256,0.108743,0.123323
221,POINT (73.86156 18.43883),0.124749,0.105965,0.108204,0.134814,0.109599,0.107620,0.102560,0.125390,0.114040,0.110374,0.102869,0.118152,0.113751,0.123299,0.125841,0.136766
222,POINT (73.86156 18.44883),0.115489,0.096877,0.098887,0.125735,0.100247,0.098433,0.093921,0.117267,0.104721,0.101253,0.094096,0.110863,0.105078,0.113934,0.116994,0.126986
242,POINT (73.87156 18.44883),0.119767,0.101639,0.103060,0.130387,0.104361,0.102957,0.099546,0.122283,0.108897,0.105919,0.099491,0.117902,0.110589,0.117948,0.122143,0.129515
202,POINT (73.85156 18.44883),0.111943,0.092950,0.095538,0.121729,0.096992,0.094743,0.089067,0.111523,0.101361,0.097393,0.089498,0.104039,0.100265,0.110680,0.112501,0.125206


In [16]:
import rasterio
import numpy as np
import geopandas as gpd

# Load Pune boundary from GeoJSON file
pune_boundary = gpd.read_file(r"C:\Users\admin\Downloads\pune-admin-wards.geojson")

# Reproject the Pune boundary to the CRS of the population density raster (e.g., UTM Zone 43N)
pune_boundary = pune_boundary.to_crs(epsg=32643)  # UTM Zone 43N for Pune

# Ensure the grid and population density raster use the same CRS
grid_crs = grid_gdf.crs
population_density_raster = rasterio.open(r"C:\Users\admin\Downloads\ind_pd_2017_1km.tif")

# Check if CRS of grid and raster match; if not, reproject the grid
if grid_crs != population_density_raster.crs:
    grid_gdf = grid_gdf.to_crs(population_density_raster.crs)

# Extract population density values for each grid point
pop_density_values = []

# Ensure the grid points are within the bounds of the raster
xmin, ymin, xmax, ymax = population_density_raster.bounds
for point in grid_gdf.geometry:
    if xmin <= point.x <= xmax and ymin <= point.y <= ymax:
        # Convert the coordinates to row, col indices for the raster
        row, col = population_density_raster.index(point.x, point.y)
        
        # Safely append population density value, checking for out-of-bounds errors
        if 0 <= row < population_density_raster.height and 0 <= col < population_density_raster.width:
            # Read the population density value at the row, col
            pop_density_value = population_density_raster.read(1)[row, col]
            pop_density_values.append(pop_density_value)
        else:
            pop_density_values.append(np.nan)  # Handle out-of-bounds points
    else:
        pop_density_values.append(np.nan)  # Handle points outside the raster bounds

# Add population density to grid GeoDataFrame
grid_gdf["population_density"] = pop_density_values

# Inspect the updated grid
grid_gdf

,geometry,distance_to_hospital,distance_to_school,distance_to_police_station,distance_to_cinema,distance_to_fuel_station,distance_to_parking,distance_to_park,distance_to_commercial_building,distance_to_residential_building,...,distance_to_industrial_landuse,distance_to_commercial_landuse,distance_to_power_generator,distance_to_transit_station,distance_to_wastewater_plant,population_density,kmeans_cluster,accessibility_score,cluster_score,dbscan_cluster
182,POINT (73.84156 18.44883),0.792265,0.688081,0.690563,0.739130,0.699647,0.683803,0.651122,0.699652,0.725521,...,0.727622,0.620544,0.750044,0.784797,0.860877,6103.210449,3,1.006716,1.594240,-1
221,POINT (73.86156 18.43883),0.906256,0.811124,0.804532,0.846448,0.812963,0.800206,0.784571,0.824965,0.837045,...,0.880368,0.734343,0.857056,0.911987,0.958473,3732.688721,2,0.566083,0.653577,-1
222,POINT (73.86156 18.44883),0.838367,0.741218,0.734009,0.786964,0.741894,0.731754,0.718482,0.771336,0.768379,...,0.826056,0.678026,0.790435,0.846179,0.887468,6846.781738,2,0.859529,0.653577,-1
242,POINT (73.87156 18.44883),0.869732,0.777849,0.765596,0.817439,0.773160,0.765459,0.761512,0.804456,0.799153,...,0.878508,0.713812,0.818989,0.884483,0.905830,7118.951172,2,0.774188,0.653577,-1
202,POINT (73.85156 18.44883),0.812370,0.711015,0.708660,0.760718,0.717165,0.704262,0.681349,0.733409,0.743630,...,0.775213,0.646766,0.767290,0.812749,0.874550,7504.191406,3,0.982869,1.594240,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,POINT (73.92156 18.58883),0.671964,0.671001,0.778519,0.733938,0.650110,0.764464,0.692818,0.708281,0.674047,...,0.596936,0.752743,0.740900,0.612798,0.603297,5904.384766,3,1.219620,1.594240,-1
316,POINT (73.90156 18.58883),0.532858,0.521357,0.643193,0.607015,0.501330,0.630089,0.544211,0.579506,0.534759,...,0.452030,0.626115,0.606643,0.468918,0.460783,5915.092285,1,1.652123,2.014455,-1
376,POINT (73.93156 18.58883),0.742592,0.746460,0.847469,0.797876,0.724984,0.833413,0.767769,0.773081,0.744851,...,0.670067,0.816437,0.809031,0.685497,0.674900,2972.632324,2,0.913105,0.653577,-1
317,POINT (73.90156 18.59883),0.562972,0.546606,0.683518,0.627829,0.523948,0.663321,0.569519,0.599065,0.565863,...,0.477435,0.644999,0.635738,0.495328,0.481421,7583.150391,1,1.632225,2.014455,-1


In [17]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Select relevant features for clustering, excluding the geometry column
features = [
    'distance_to_hospital', 'distance_to_school', 'distance_to_police_station', 
    'distance_to_cinema', 'distance_to_fuel_station', 'distance_to_parking', 
    'distance_to_park', 'distance_to_commercial_building', 
    'distance_to_residential_building', 'distance_to_retail', 'distance_to_residential_landuse', 
    'distance_to_industrial_landuse', 'distance_to_commercial_landuse', 
    'distance_to_power_generator', 'distance_to_transit_station', 
    'distance_to_wastewater_plant', 'population_density'
]

# Ensure all selected features are numeric
for feature in features:
    if grid_gdf[feature].dtype not in [np.float64, np.int64]:
        grid_gdf[feature] = pd.to_numeric(grid_gdf[feature], errors='coerce')

# Fill any NaN values in the features with the column mean
grid_gdf[features] = grid_gdf[features].fillna(grid_gdf[features].mean())

In [18]:
# Normalize the features
scaler = MinMaxScaler()
grid_gdf[features] = scaler.fit_transform(grid_gdf[features])

In [19]:
# Define number of clusters (you can experiment with different values)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0)

# Fit the model on the normalized data and add the cluster labels
grid_gdf['kmeans_cluster'] = kmeans.fit_predict(grid_gdf[features])

# Verify the clustering output
print(grid_gdf[['geometry', 'kmeans_cluster'] + features].head())

C:\Users\admin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


                      geometry  kmeans_cluster  distance_to_hospital  \
182  POINT (73.84156 18.44883)               4              0.792265   
221  POINT (73.86156 18.43883)               3              0.906256   
222  POINT (73.86156 18.44883)               4              0.838367   
242  POINT (73.87156 18.44883)               3              0.869732   
202  POINT (73.85156 18.44883)               4              0.812370   

     distance_to_school  distance_to_police_station  distance_to_cinema  \
182            0.688081                    0.690563            0.739130   
221            0.811124                    0.804532            0.846448   
222            0.741218                    0.734009            0.786964   
242            0.777849                    0.765596            0.817439   
202            0.711015                    0.708660            0.760718   

     distance_to_fuel_station  distance_to_parking  distance_to_park  \
182                  0.699647             0.

In [20]:
# Define weights for each feature to create a score (tune these based on importance)
weights = {
    'population_density': 1.5,
    'distance_to_fuel_station': 1.2,
    'distance_to_transit_station': 1.0,
    'distance_to_commercial_building': 0.8,
}

# Calculate the score for each point based on the weighted sum of important features
grid_gdf['accessibility_score'] = (
    weights['population_density'] * grid_gdf['population_density'] +
    weights['distance_to_fuel_station'] * (1 - grid_gdf['distance_to_fuel_station']) +
    weights['distance_to_transit_station'] * (1 - grid_gdf['distance_to_transit_station']) +
    weights['distance_to_commercial_building'] * (1 - grid_gdf['distance_to_commercial_building'])
)

# Calculate the mean score for each cluster
cluster_scores = grid_gdf.groupby('kmeans_cluster')['accessibility_score'].mean()
grid_gdf['cluster_score'] = grid_gdf['kmeans_cluster'].map(cluster_scores)

In [21]:
# Optional: Inspect clusters in KMeans
print("Number of clusters found by KMeans:", len(set(grid_gdf['kmeans_cluster'])))

Number of clusters found by KMeans: 5


In [22]:
from sklearn.metrics import davies_bouldin_score

# Calculate the Davies-Bouldin Index (DBI)
dbi_score = davies_bouldin_score(grid_gdf[features], grid_gdf['kmeans_cluster'])

print("Davies-Bouldin Index (DBI) for the KMeans Clustering:", dbi_score)

Davies-Bouldin Index (DBI) for the KMeans Clustering: 0.8804619442598197


In [23]:
import folium

# Create a Folium map centered on Pune
pune_map = folium.Map(location=[18.5204, 73.8567], zoom_start=12)

# Add clusters to the map
for _, row in grid_gdf.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),
        radius=5,
        color="blue" if row['cluster_score'] > cluster_scores.mean() else "red",
        fill=True,
        fill_opacity=0.6,
        popup=f"Cluster Score: {row['cluster_score']:.2f}"
    ).add_to(pune_map)

# Display the map
pune_map.save("high_priority_ev_clusters_map.html")
pune_map

In [24]:
# Define a threshold for high-priority locations (e.g., above the mean cluster score)
threshold = cluster_scores.mean()

# Filter high-priority locations (above the threshold)
high_priority_locations = grid_gdf[grid_gdf['cluster_score'] > threshold]

# Plot high-priority locations on the map
for _, row in high_priority_locations.iterrows():
    folium.CircleMarker(
        location=(row.geometry.y, row.geometry.x),
        radius=5,
        color="green",  # Use green for high-priority EV locations
        fill=True,
        fill_opacity=0.8,
        popup=f"Cluster Score: {row['cluster_score']:.2f}"
    ).add_to(pune_map)

# Save and display the map with high-priority EV station locations
pune_map

In [25]:
from sklearn.metrics import davies_bouldin_score

# Calculate the Davies-Bouldin Index (DBI)
dbi_score = davies_bouldin_score(grid_gdf[features], grid_gdf['kmeans_cluster'])

print("Davies-Bouldin Index (DBI) for the KMeans Clustering:", dbi_score)

Davies-Bouldin Index (DBI) for the KMeans Clustering: 0.8804619442598197
